In [ ]:
import importlib
import datetime
import random
import collections
import math

import haversine
import folium
import pandas as pd
import numpy as np
from nltk.cluster.kmeans import KMeansClusterer
from sklearn.metrics import pairwise_distances

from projects.common import geo
from projects.common.nile import dates, test_utils
from projects.cpo_model import project_config, default_values, common

In [ ]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 100

In [ ]:
_ = importlib.reload(default_values)
_ = importlib.reload(project_config)
_ = importlib.reload(common)
_ = importlib.reload(test_utils)
_ = importlib.reload(geo)

In [ ]:
cluster = project_config.get_project_cluster(
    parallel_operations_limit=5,
    pool='taxi-delivery',
    extra_requirements=[],
)

### Define constants

### Define functions

In [ ]:
def convert_to_datetime(dttm_str):
    return datetime.datetime.strptime(dttm_str, '%Y-%m-%d %H:%M:%S')

def choose_random_time_in_interval(dttm_left, dttm_right):
    delta = (dttm_right - dttm_left).total_seconds()
    return dttm_left + datetime.timedelta(seconds=random.randint(0, delta))

def parse_gps_str(gps_str):
    return tuple(float(c) for c in gps_str.split(','))

def assign_day(dttm, month, day):
    return dttm.replace(month=month, day=day)

### Get and prepare dataframes

In [ ]:
# lavka_places_df = test_utils.decode_dataframe(
#     cluster
#     .read('//home/taxi-delivery/analytics/dev/cpo_model/lavka_places_moscow')
#     .as_dataframe()
# )
# lavka_places_df = lavka_places_df[pd.isnull(lavka_places_df['disable_details_status'])].reset_index()

In [ ]:
lavka_places_df = test_utils.decode_dataframe(
    cluster
    .read('//home/lavka/production/replica/postgres/wms/stores')
    .as_dataframe()
)
lavka_places_df = lavka_places_df[
    (lavka_places_df['status']=='active')
    & (lavka_places_df['type']=='lavka')
    & (lavka_places_df['region_id']!=131)
]

In [ ]:
lavka_places_df.groupby('region_id').count()

In [ ]:
# additional_pp = pd.read_csv('./analytics/real_pp_coordinates_moscow.csv', index_col=0)

In [ ]:
# additional_pp.head()

In [ ]:
market_orders_df = test_utils.decode_dataframe(
    cluster
    .read('//home/taxi-delivery/analytics/dev/cpo_model/m+_city_orders_2020-11-01_2021-01-10')
    .as_dataframe()
)
market_orders_df = market_orders_df[~pd.isnull(market_orders_df['final_delivery_point_gps'])]

In [ ]:
market_orders_df.head(2)

In [ ]:
# Pickup Points.
# pickup_points_df = pd.DataFrame()
# pickup_points_df['pickup_point_id'] = np.arange(len(lavka_places_df))
# pickup_points_df['lat'] = lavka_places_df['location_lat']
# pickup_points_df['lon'] = lavka_places_df['location_lon']
# pickup_points_df['take_start_tm'] = '06:00:00'
# pickup_points_df['take_end_tm'] = '07:00:00'
# pickup_points_df['address'] = lavka_places_df['address_full']

In [ ]:
# Pickup points.
pickup_points_df = pd.DataFrame()
pickup_points_df['pickup_point_id'] = lavka_places_df['store_id']
pickup_points_df['lat'] = lavka_places_df['location'].apply(
    lambda x: float(x.strip('(').strip(')').split(',')[1])
)
pickup_points_df['lon'] = lavka_places_df['location'].apply(
    lambda x: float(x.strip('(').strip(')').split(',')[0])
)
pickup_points_df['region_id'] = lavka_places_df['region_id']
pickup_points_df['take_start_tm'] = '06:00:00'
pickup_points_df['take_end_tm'] = '07:00:00'
pickup_points_df['address'] = lavka_places_df['address']

In [ ]:
# add_pickup_points_df = pd.DataFrame()
# add_pickup_points_df['pickup_point_id'] = 1000 + np.arange(len(additional_pp))
# add_pickup_points_df['lat'] = additional_pp['lat'].values
# add_pickup_points_df['lon'] = additional_pp['lon'].values
# add_pickup_points_df['take_start_tm'] = '06:00:00'
# add_pickup_points_df['take_end_tm'] = '07:00:00'
# add_pickup_points_df['address'] = None

In [ ]:
# pickup_points_df = pd.concat([pickup_points_df, add_pickup_points_df], axis=0)

In [ ]:
# pickup_points_df.head()

In [ ]:
market_orders_df['delivery_type'].value_counts()

In [ ]:
market_orders_df = market_orders_df[market_orders_df['delivery_type']=='Курьерская доставка']
market_orders_df = market_orders_df[market_orders_df['order_physical_weight']<=10]
market_orders_df = market_orders_df[market_orders_df['order_volume'] <= (0.5*0.5*0.5)]

In [ ]:
market_orders_df['original_delivery_to_date'] = market_orders_df[
    'original_delivery_to_datetime'
].apply(lambda x: x[:10])

In [ ]:
market_orders_df.groupby(['delivery_region_id']).count()['creation_datetime']

In [ ]:
market_orders_df['original_delivery_from_datetime'] = (
    market_orders_df['original_delivery_from_datetime'].apply(
        lambda x: x[:10] + ' ' + '09:00:00' if x[11:] < '09:00:00' else x
    ).apply(
        lambda x: assign_day(convert_to_datetime(x), 11, 2)
    )
)
market_orders_df['original_delivery_to_datetime'] = (
    market_orders_df['original_delivery_to_datetime'].apply(
        lambda x: x[:10] + ' ' + '23:00:00' if x[11:] > '23:00:00' else x
    ).apply(
        lambda x: assign_day(convert_to_datetime(x), 11, 2)
    )
)

In [ ]:
# Market Orders.
market_deliveries_df = pd.DataFrame()
market_deliveries_df['delivery_region_id'] = market_orders_df['delivery_region_id']
market_deliveries_df['service_name'] = ['market']*len(market_orders_df)
market_deliveries_df['weight'] = market_orders_df['order_physical_weight'].fillna(
    np.mean(market_orders_df['order_physical_weight'])
)
market_deliveries_df['volume'] = None
market_deliveries_df['delivery_lat'] = (
    market_orders_df['final_delivery_point_gps'].apply(lambda x: parse_gps_str(x)[1])
)
market_deliveries_df['delivery_lon'] = (
    market_orders_df['final_delivery_point_gps'].apply(lambda x: parse_gps_str(x)[0])
)
market_deliveries_df['warehouse_lat'] = None
market_deliveries_df['warehouse_lon'] = None
market_deliveries_df['warehouse_id'] = None
market_deliveries_df['delivery_plan_dttm'] = market_orders_df.apply(
    lambda x: choose_random_time_in_interval(
        x['original_delivery_from_datetime'],
        x['original_delivery_to_datetime']
    ),
    axis=1
)
market_deliveries_df['order_id_source'] = market_orders_df['order_id'].astype(int)
market_deliveries_df['warehouse_ready_dttm'] = None
market_deliveries_df = market_deliveries_df[~pd.isnull(market_deliveries_df['delivery_lat'])]
market_deliveries_df = market_deliveries_df[~pd.isnull(market_deliveries_df['delivery_lon'])]


In [ ]:
market_deliveries_df.head()

### Run simulation

In [ ]:
CITY_VOLUME_MAP = {
     2: 6050,
     35: 995,
     38: 403,
     39: 978,
     43: 690,
     47: 898,
     50: 517,
     51: 717,
     54: 541,
     56: 243,
     62: 118,
     65: 403,
     66: 117,
     172: 535,
     193: 849,
     213: 15189
}

CITY_MAP = {
    'Москва': 213,
    'Санкт-Петербург': 2,
    'Новосибирск': 65,
    'Екатеринбург': 54,
    'Нижний Новгород': 47,
    'Казань': 43,
    'Челябинск': 56,
    'Омск': 66,
    'Самара': 51,
    'Ростов-на-Дону': 39,
    'Уфа': 172,
    'Красноярск': 62,
    'Пермь': 50,
    'Воронеж': 193,
    'Волгоград': 38,
    'Краснодар': 35,
}

In [ ]:
CITY = 'Екатеринбург'
COEFFICIENT = 2
SLA_SEC = 10*60
SUCCESS_DELIVERIES_FRAC = 0.9
order_cnt = int(CITY_VOLUME_MAP[CITY_MAP[CITY]] * COEFFICIENT)

In [ ]:
deliveries_df = pd.concat(
    [
     market_deliveries_df[market_deliveries_df['delivery_region_id']==CITY_MAP[CITY]].sample(n=order_cnt, random_state=7)],
    axis=0
)

In [ ]:
city_coordinates = (deliveries_df['delivery_lat'].mean(), deliveries_df['delivery_lon'].mean())

In [ ]:
deliveries_df.head()

In [ ]:
sim = common.Simulation(sla_seconds=10*60, sh_cost=250, pp_work_time=14)
sim.initialize_deliveries(deliveries_df)

In [ ]:
# simulation_dict = {}
# for key, value in simulation_paramaters.items():
#     simulation_dict[key] = {}
    
#     deliveries_df = pd.concat(
#         [
#          market_deliveries_df.sample(n=value['n_market_orders'], random_state=7)],
#         axis=0
#     )
#     print(key)
#     print(deliveries_df.shape)
    
#     sim = common.Simulation(sla_seconds=10*60, sh_cost=250, pp_work_time=14)
#     simulation_dict[key]['simulation'] = sim
    
#     sim.initialize_simulation(deliveries_df, pickup_points_df)
#     sim.assign_objects()
#     sim.batch_deliveries_pp_to_user()
#     sim.pay_couriers()
#     print('\n')

### Optimal geo of pp

In [ ]:
def find_optimal_pp_geo(radius, deliveries, coverage_threshold=0.8):
    X = np.array(
        [
            [delivery.location_lat, delivery.location_lon]
            for delivery in deliveries
        ],
    )

    # Choose number of clusters.
    num_clusters = math.ceil(len(deliveries) / 120)
    
    success_deliveries_frac = 0
    
    # Optimize pickup points.
    while success_deliveries_frac < coverage_threshold:
        kclusterer = KMeansClusterer(num_clusters,
                                     distance=haversine.haversine,
                                     repeats=1,
                                     avoid_empty_clusters=True,)
        assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

        failed_deliveries_cnt = 0
        ac_points_in_radius = {}
        for ac, d in zip(assigned_clusters, deliveries):
            dist_ac = haversine.haversine(
                kclusterer.means()[ac], (d.location_lat, d.location_lon),
                'm'
            )
            if dist_ac > radius:
                failed_deliveries_cnt += 1

        success_deliveries_frac = (len(deliveries_df) - failed_deliveries_cnt) / len(deliveries_df)
        if success_deliveries_frac < coverage_threshold:
            num_clusters += 1
            print('increased num clusters to {}, success deliveries frac {}'.format(
                num_clusters, success_deliveries_frac
            ))

    # Get pp df.
    pp_addresses = pd.DataFrame(
        {
            'pp_id': list(range(len(kclusterer.means()))),
            'lat': [c[0] for c in kclusterer.means()],
            'lon': [c[1] for c in kclusterer.means()],
            'deliveries_cnt': [c[1] for c in sorted(collections.Counter(assigned_clusters).items(),
                                               key=lambda x: x[0])]
        }
    )
    
    return {
        'pp_df': pp_addresses,
        'coverage': success_deliveries_frac,
        'total_deliveries_cnt': len(deliveries),
    }

In [ ]:
optimal_geo_dict = {}
for city in (CITY_MAP.keys() - set(['Москва', 'Нижний Новгород', 'Санкт-Петербург'])):
    
    order_cnt = int(CITY_VOLUME_MAP[CITY_MAP[city]] * COEFFICIENT)
    deliveries_df = pd.concat(
        [
         market_deliveries_df[market_deliveries_df['delivery_region_id']==CITY_MAP[city]].sample(n=order_cnt, random_state=7)],
        axis=0
    )
    sim = common.Simulation(sla_seconds=10*60, sh_cost=250, pp_work_time=14)
    sim.initialize_deliveries(deliveries_df)
    
    print(city, order_cnt)
    
    optimal_geo_dict[city] = find_optimal_pp_geo(
        radius=(SLA_SEC / 60) * 132.1,
        deliveries=[d for d in sim.deliveries if not d.delivered_flg],
        coverage_threshold=0.8,
    )
    
    optimal_geo_dict[city]['city_coordinates'] = (
        deliveries_df['delivery_lat'].mean(), deliveries_df['delivery_lon'].mean()
    )
    optimal_geo_dict[city]['sim'] = sim
    print()

In [ ]:
for city in set(['Москва', 'Нижний Новгород', 'Санкт-Петербург']):
    
    lavkas_df = pickup_points_df[pickup_points_df['region_id'] == CITY_MAP[city]]
    
    order_cnt = int(CITY_VOLUME_MAP[CITY_MAP[city]] * COEFFICIENT)
    deliveries_df = pd.concat(
        [
         market_deliveries_df[market_deliveries_df['delivery_region_id']==CITY_MAP[city]].sample(n=order_cnt, random_state=7)],
        axis=0
    )
    sim = common.Simulation(sla_seconds=10*60, sh_cost=250, pp_work_time=14)
    sim.initialize_simulation(deliveries_df, lavkas_df)
    sim.assign_objects()
    sim.batch_deliveries_pp_to_user()
    
    print(city, order_cnt)
    
    optimal_geo_dict[city] = find_optimal_pp_geo(
        radius=(SLA_SEC / 60) * 132.1,
        deliveries=[d for d in sim.deliveries if not d.delivered_flg],
        coverage_threshold=0.8,
    )
    
    optimal_geo_dict[city]['city_coordinates'] = (
        deliveries_df['delivery_lat'].mean(), deliveries_df['delivery_lon'].mean()
    )
    optimal_geo_dict[city]['sim'] = sim
    print()

In [ ]:
[(k, v['coverage'], v['total_deliveries_cnt']) for k, v in optimal_geo_dict.items()]

In [ ]:
# p = optimal_geo_dict['Москва']['sim'].pickup_points_dict['5ff2405172bc483d9fc8a876bedd9384000300010000']

In [ ]:
df_list_old_lavkas = []
for city in set(['Москва', 'Нижний Новгород', 'Санкт-Петербург']):
    sim = optimal_geo_dict[city]['sim']
    
    pp_list = []
    for pp_id, pp in sim.pickup_points_dict.items():
        pickup_point_deliveries = list(
            sorted(
                filter(
                    lambda delivery: delivery.pickup_point_id == pp.id
                    and delivery.delivered_flg,
                    sim.deliveries,
                ),
                key=lambda delivery: delivery.delivery_plan_dttm,
            ),
        )
        pp.deliveries_cnt = len(pickup_point_deliveries)
        pp_list.append(({'pp_id': pp.id, 'lat': pp.lat, 'lon': pp.lon,
                         'pp_deliveries_cnt': pp.deliveries_cnt,
                         'lavka_type': 'old'}))
        

    
    pp_df = pd.DataFrame(pp_list)
    pp_df['city'] = [city] * len(sim.pickup_points_dict)
    pp_df['coverage'] = round(
        len([d for d in sim.deliveries if d.delivered_flg])
        / len(sim.deliveries),
        2
    )
    pp_df['total_deliveries_cnt'] = len(sim.deliveries)
    df_list_old_lavkas.append(pp_df)

In [ ]:
pd.concat(df_list_old_lavkas)

In [ ]:
pd.concat(df_list_old_lavkas).to_excel('./msk_nn_spb_old_lavka_goals_v0.xlsx')

In [ ]:
df_list = []
for k, v in optimal_geo_dict.items():
    v['pp_df']['city'] = k
    v['pp_df']['coverage'] = round(v['coverage'], 2)
    v['pp_df']['lavka_type'] = 'new'
    v['pp_df']['total_deliveries_cnt'] = len(
        [d for d in v['sim'].deliveries if not d.delivered_flg]
    )
    df_list.append(v['pp_df'])

In [ ]:
pd.concat(df_list).to_excel('./1m+_lavka_goals_v0.xlsx')

In [ ]:
# pp_addresses.to_excel('./pp_addresses_krasnodar_estimate.xlsx')

### Real PP

In [ ]:
real_pp_df = pd.read_excel('./import_data/pp_list.xlsx', skiprows=3)
real_pp_df = real_pp_df[(real_pp_df['Населённый пункт']=='г Москва')
& (real_pp_df['Тип пункта']=='ПВЗ')][['Широта', 'Долгота']].drop_duplicates(subset=('Широта', 'Долгота')).reset_index(
).rename(
    {'index': 'real_pp_id', 'Широта': 'lat', 'Долгота': 'lon'}, axis=1
)

In [ ]:
pp_addresses.to_excel('./pp_addresses_moscow_estimate.xlsx')

### Draw orders on map

In [ ]:
YA_MAPS_TILES = 'http://vec02.maps.yandex.net/tiles?l=map&v=2.2.3&x={x}&y={y}&z={z}'

In [ ]:
f_map = folium.Map(
    location=city_coordinates, zoom_start=10, tiles=YA_MAPS_TILES, max_zoom=20,
    attr='My Data Attribution', crs='EPSG3395'
)

In [ ]:
for d in sim.deliveries:
    c = 'red'
#     if d.delivered_flg:
#         if d.pickup_point_id >= 1000:
#             c = 'blue'
#         else:
#             c = 'green'
    folium.CircleMarker(
        location=(d.location_lat, d.location_lon),
        radius=0.0005,
        color=c,
        fill=True,
        weight=0.1,
    ).add_to(f_map)
    
# for row in pp_addresses.itertuples():
#     c = 'black'
#     folium.CircleMarker(
#         location=(row.lat, row.lon),
#         radius=3,
#         color=c,
#         fill=True,
#         weight=3,
#     ).add_to(f_map)
    
# for row in real_pp_df.itertuples():
#     c = 'orange'
#     folium.CircleMarker(
#         location=(row.lat, row.lon),
#         radius=3,
#         color=c,
#         fill=True,
#         weight=3,
#     ).add_to(f_map)

In [ ]:
f_map.save('./krasnodar_test.html')